I want to maximize the flow of valves. Each valve can be represented in a network as a node, I can perform a search, but there are several elements to account for. I will spend time opening a valve, thus it can be more efficient to skip opening one and moving on. The search algorithm will have the oportunity to visit the same nodes as many times as possible, but will only be able to open a valve once. The flow is dependent on time, so the maximizing of flow is not only dependent on visiting the nodes with the highest value, but also visiting them as soon as possible.

In [7]:
import networkx as nx
import regex as re

In [61]:
data = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II""".split("\n")

In [62]:
data

['Valve AA has flow rate=0; tunnels lead to valves DD, II, BB',
 'Valve BB has flow rate=13; tunnels lead to valves CC, AA',
 'Valve CC has flow rate=2; tunnels lead to valves DD, BB',
 'Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE',
 'Valve EE has flow rate=3; tunnels lead to valves FF, DD',
 'Valve FF has flow rate=0; tunnels lead to valves EE, GG',
 'Valve GG has flow rate=0; tunnels lead to valves FF, HH',
 'Valve HH has flow rate=22; tunnel leads to valve GG',
 'Valve II has flow rate=0; tunnels lead to valves AA, JJ',
 'Valve JJ has flow rate=21; tunnel leads to valve II']

In [63]:
def valve_node_from_input(data):
    valve_flow = re.findall("(?:Valve\s)(\w+)(?:.+=)(\d+)",data)
    return valve_flow[0]

def edge_from_input(data):
    nodes = re.findall("[A-Z]{2}",data)
    edges = []
    for node in nodes[1:]:
        edge = (nodes[0],node)
        edges.append(edge)
    return edges

In [64]:
valve_node = [valve_node_from_input(node) for node in data]

In [78]:
nodes = {}
for node, flow in valve_node:
    nodes[node] = {"flow": int(flow), "visited":False}

In [79]:
G = nx.Graph()
for node in nodes:
    G.add_node(node)

In [65]:
edge_list = []
for valve_input in data:
    edges = edge_from_input(valve_input)
    edge_list.extend(edges)

In [66]:
edge_list

[('AA', 'DD'),
 ('AA', 'II'),
 ('AA', 'BB'),
 ('BB', 'CC'),
 ('BB', 'AA'),
 ('CC', 'DD'),
 ('CC', 'BB'),
 ('DD', 'CC'),
 ('DD', 'AA'),
 ('DD', 'EE'),
 ('EE', 'FF'),
 ('EE', 'DD'),
 ('FF', 'EE'),
 ('FF', 'GG'),
 ('GG', 'FF'),
 ('GG', 'HH'),
 ('HH', 'GG'),
 ('II', 'AA'),
 ('II', 'JJ'),
 ('JJ', 'II')]

In [81]:
for edge in edge_list:
    G.add_edge(edge[0],edge[1])

In [82]:
[*G.neighbors("AA")]

['DD', 'II', 'BB']

In [83]:
simple_paths = []
for node in G.nodes():
    if len([*G.neighbors(node)])==1:
        paths = [*nx.all_simple_paths(G, source= "AA",target = node)]
        simple_paths.extend(paths)

In [84]:
simple_paths

[['AA', 'DD', 'EE', 'FF', 'GG', 'HH'],
 ['AA', 'BB', 'CC', 'DD', 'EE', 'FF', 'GG', 'HH'],
 ['AA', 'II', 'JJ']]

In [113]:
G_tree = nx.bfs_tree(G, "AA")

In [114]:
leaf = [x for x in G_tree.nodes() if G_tree.out_degree(x)==0 and G_tree.in_degree(x)==1]

In [115]:
simple_paths = []
for node in leaf:
    paths = [*nx.all_simple_paths(G_tree, source= "AA",target = node)]
    simple_paths.extend(paths)

In [116]:
simple_paths

[['AA', 'BB'],
 ['AA', 'DD', 'CC'],
 ['AA', 'II', 'JJ'],
 ['AA', 'DD', 'EE', 'FF', 'GG', 'HH']]

In [117]:
nodes["AA"]["flow"]

0

In [144]:
visited_nodes = []
minutes = 30
current_node = "AA"
paths_sort_generated = []


for i in range(10):
    # First i generate a bfs-tree from the current node
    G_tree = nx.bfs_tree(G, current_node)
    
    # Then i generate simple paths to all leaf nodes
    leaf = [x for x in G_tree.nodes() if G_tree.out_degree(x)==0 and G_tree.in_degree(x)==1]
    simple_paths = []
    for node in leaf:
        paths = [*nx.all_simple_paths(G_tree, source= current_node,target = node)]
        simple_paths.extend(paths)
    
    # I calculate the flow if every node is visited and opened
    paths = {}
    for s_path in simple_paths:
        flow = 0
        for node in reversed(s_path[1:]):
            if node in visited_nodes:
                flow -= 2
                continue
            minutes_path = minutes - len(s_path[1:])
            flow += minutes_path-1 * nodes[node]["flow"]
        paths[flow/len(s_path[1:])] = s_path[1:]
    
    # I sort the paths and pick the first node of the path that has the highest value
    paths_sort = sorted(paths.items(),reverse=True)
    current_node = paths_sort[0][1][0]
    visited_nodes.append(current_node)
    paths_sort_generated.append(paths_sort)
    

In [145]:
visited_nodes

['II', 'AA', 'DD', 'CC', 'DD', 'EE', 'FF', 'GG', 'HH', 'GG']

In [146]:
paths_sort_generated

[[(17.5, ['II', 'JJ']),
  (17.0, ['DD', 'CC']),
  (16.0, ['DD', 'EE', 'FF', 'GG', 'HH'])],
 [(21.5, ['AA', 'BB']),
  (19.666666666666668, ['AA', 'DD', 'CC']),
  (16.5, ['AA', 'DD', 'EE', 'FF', 'GG', 'HH']),
  (8.0, ['JJ'])],
 [(17.0, ['DD', 'CC']),
  (16.0, ['DD', 'EE', 'FF', 'GG', 'HH']),
  (2.5, ['II', 'JJ'])],
 [(27.0, ['CC']),
  (19.75, ['EE', 'FF', 'GG', 'HH']),
  (6.5, ['AA', 'BB']),
  (0.6666666666666666, ['AA', 'II', 'JJ'])],
 [(14.6, ['DD', 'EE', 'FF', 'GG', 'HH']), (3.5, ['BB', 'AA', 'II', 'JJ'])],
 [(19.75, ['EE', 'FF', 'GG', 'HH']),
  (6.5, ['AA', 'BB']),
  (0.6666666666666666, ['AA', 'II', 'JJ']),
  (-2.0, ['CC'])],
 [(19.666666666666668, ['FF', 'GG', 'HH']),
  (3.3333333333333335, ['DD', 'AA', 'BB']),
  (-0.25, ['DD', 'AA', 'II', 'JJ']),
  (-2.0, ['DD', 'CC'])],
 [(17.0, ['GG', 'HH']),
  (1.75, ['EE', 'DD', 'AA', 'BB']),
  (-0.8, ['EE', 'DD', 'AA', 'II', 'JJ']),
  (-2.0, ['EE', 'DD', 'CC'])],
 [(7.0, ['HH']),
  (0.8, ['FF', 'EE', 'DD', 'AA', 'BB']),
  (-1.1666666666666667